<a href="https://colab.research.google.com/github/aksekhar/Signal-Strength-or-quality-model./blob/master/Signal_Strength_or_quality_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hellping Class and functions

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import model_selection
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import warnings
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Input, Dropout,BatchNormalization

import random
from tensorflow.keras import backend
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)
warnings.filterwarnings("ignore")

In [ ]:
def mount_drive(fpath):
  try:
    from google.colab import drive
    drive.mount("/content/drive/", force_remount=True)
    google_drive_prefix = "/content/drive/My Drive"
    data_path = f"{google_drive_prefix}/{fpath}"
    print(data_path)
    return data_path
  except ModuleNotFoundError:
    data_prefix = f"{fpath}"

In [ ]:
def read_file(file_name):
  print(f"reding the file: {file_name}")
  try:
    file_type = file_name.split('.')[1]
    print(file_type)
    if file_type == 'csv':
      df = pd.read_csv(signal_file_path)
    else:
      pass
    return df
  except Exception as e:
    print("data write")
    raise e

In [ ]:
def verify_sample_data(df,records):
  if not(records):
    records = 5
  print(f"verifying first {records} records and last {records} records")
  return pd.concat([df.head(records),df.tail(records)])

In [ ]:
def get_percentage_of_missing_values(df):
  print("Checking for missing values and print percentage for each attribute")
  return (round(df.isnull().sum() / (df.isnull().count())*100))

In [ ]:
import matplotlib.pyplot as plt

def plot_distribution(data,variable_name):

  target_variable = data[variable_name]
  plt.hist(target_variable)
  plt.xlabel(f"{variable_name}")
  plt.ylabel("Frequency")
  plt.title("Distribution of Target Variable")
  plt.show()

In [ ]:
def plot_tring_accuracy(history,t_lebel,v_level,y_level):

  plt.figure(figsize=(5, 3))
  plt.plot(history.history['accuracy'], color='b', label=t_lebel)
  plt.plot(history.history['val_accuracy'], color='r', label=v_level)
  plt.title(f"{t_lebel} and {v_level}")
  plt.xlabel('Epoch')
  plt.ylabel(y_level)
  plt.legend()
  plt.show()

# 1. Data import and Understanding

In [ ]:
signal_file_path = mount_drive("dataset/NN Project Data - Signal.csv")

***a. Read the ‘Signals.csv’ as DataFrame and import required libraries.***

In [ ]:
signalDf = read_file(signal_file_path)
verify_sample_data(signalDf,3)

In [ ]:
signalDf.info()

- The dataset consists of **1599 entries** (non-null values).
- There are **12 columns** in total, labeled as `Parameter 1` through `Parameter 11` and `Signal_Strength`.
- All `Parameter` columns contain **floating-point values** (float64), which suggests they could be measurements or scores of some kind.
- The `Signal_Strength` column contains **integer values** (int64), which could be a count, a rating, or some other discrete value.
- There are **no missing values** in the dataset, as indicated by the 'non-null' count for each column.


**b. Check for missing values and print percentage for each attribute.**

In [ ]:
get_percentage_of_missing_values(signalDf)

1. **Parameter 1**: There are **96** unique values.
2. **Parameter 2**: There are **143** unique values.
3. **Parameter 3**: There are **80** unique values.
4. **Parameter 4**: There are **91** unique values.
5. **Parameter 5**: There are **153** unique values.
6. **Parameter 6**: There are **60** unique values.
7. **Parameter 7**: There are **144** unique values.
8. **Parameter 8**: There are **436** unique values.
9. **Parameter 9**: There are **89** unique values.
10. **Parameter 10**: There are **96** unique values.
11. **Parameter 11**: There are **65** unique values.

The signal strength target variables is **6** unique values.


**c. Check for presence of duplicate records in the dataset and impute with appropriate method**

In [ ]:
signalDf.nunique()

**`d. Visualise distribution of the target variable`**

In [ ]:
signalDf["Signal_Strength"].describe().T

In [ ]:
plot_distribution(signalDf,"Signal_Strength")

- **Mean**: The mean (average) value of the target variable is approximately **5.64**.
- **Standard Deviation**: The standard deviation measures the spread or variability of the target variable. In this case, it's approximately **0.81**.
- **Minimum**: The minimum value of the target variable is **3**.
- **25th Percentile (Q1)**: This represents the value below which 25% of the data falls. It's **5** in this case.
- **Median (50th Percentile)**: The median (also known as the 50th percentile) is the middle value of the dataset when sorted. It's **6** here.
- **75th Percentile (Q3)**: This represents the value below which 75% of the data falls. It's also **6**.

From this information, we can infer that the majority of the target variable values are around **5** or **6**, with some variability. The distribution appears to be relatively centered.

**e. Share insights from the initial data analysis**

In [ ]:

signalDf.describe().T

1. **Parameter 1**: The data is centered around 8.32 with a spread of 1.74. The wide range suggests diverse data points.

2. **Parameter 2**: The average value is 0.53, indicating that most data points are close to this value. However, the maximum value of 1.58 suggests the presence of higher values as well.

3. **Parameter 3**: With a mean of 0.27 and a maximum value of 1, it appears that the data points are skewed towards the lower end.

4. **Parameter 4**: The data has a fairly large spread (std: 1.41) around the mean (2.54), indicating variability in the data points.

5. **Parameter 5**: The data is tightly clustered around the mean (0.087), as indicated by the small standard deviation (0.047).

6. **Parameter 6**: The large standard deviation (10.46) relative to the mean (15.87) suggests a wide spread in the data.

7. **Parameter 7**: The data points are spread out over a large range (from 6 to 289), indicating high variability.

8. **Parameter 8**: The data is tightly clustered around the mean (0.997), suggesting low variability.

9. **Parameter 9**: The data points are relatively close to the mean (3.31), indicating moderate variability.

10. **Parameter 10**: The data has a moderate spread (std: 0.17) around the mean (0.658), indicating some variability.

11. **Parameter 11**: The data points are fairly close to the mean (10.42), suggesting moderate variability.

12. **Signal Strength**: The average signal strength is 5.64 with a moderate spread (std: 0.81), indicating that the signal strength varies moderately across observations.

# 2. Data preprocessing

**a.Split the data into X & Y**



In [ ]:
X = signalDf.drop(['Signal_Strength'],axis=1)
Y = signalDf[['Signal_Strength']]

In [ ]:
YC = to_categorical(Y)

**b.Split the data into train & test with 70:30 proportion.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,YC, test_size = 0.3, random_state = 42,stratify = Y)

In [ ]:
print("Number of training samples:", X_train.shape[0])
print("Number of testing samples:", X_test.shape[0])

**C. Print shape of all the 4 variables and verify if train and test data is in sync**

In [ ]:
# Print shapes of all 4 variables
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Verify if train and test data is in sync
if X_train.shape[0] == y_train.shape[0] and X_test.shape[0] == y_test.shape[0]:
  print("Train and test data are in sync")
else:
  print("Train and test data are not in sync")

**D. Normalise the train and test data with appropriate method**

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
print(X_train_normalized.shape);
print(X_test_normalized.shape);

**E. Transform Labels into format acceptable by Neural Network**

# 3. Model Training & Evaluation using Neural Network

In [ ]:
backend.clear_session()
np.random.seed(42)
import random
random.seed(42)
tf.random.set_seed(42)

**A. Design a Neural Network to train a classifier**

In [ ]:
model = Sequential()
model.add(Dense(11, activation='relu',input_dim = X_train_normalized.shape[1]))
model.add(Dense(21, activation='relu'))
model.add(Dense(9, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

**B. Train the classifier using previously designed Architecture**

In [ ]:
history = model.fit(x=X_train_normalized, y=y_train, batch_size=20, epochs=50, validation_data=(X_test_normalized, y_test))

**C. Plot 2 separate visuals.**

  *i. Training Loss and Validation Loss*

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 3))
plt.plot(history.history['loss'], color='b', label='Training Loss')
plt.plot(history.history['val_loss'], color='r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

The graph shows training and validation loss over epochs. The training loss decreases sharply then plateaus, while the validation loss decreases more gradually and slightly increases towards the end, indicating possible overfitting. Strategies like early stopping, dropout, or regularization could be used to improve the model's performance on unseen data.

 *ii. Training Accuracy and Validation Accuracy*

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 3))
plt.plot(history.history['accuracy'], color='b', label='Training accuracy')
plt.plot(history.history['val_accuracy'], color='r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()

The graph shows the training and validation accuracy of a model over 50 epochs. Both accuracies increase until the 10th epoch, then plateau. The consistently higher training accuracy may indicate overfitting.

**D. Design new architecture/update existing architecture in attempt to improve the performance of the model.**

In [ ]:
backend.clear_session()
np.random.seed(42)
import random
random.seed(42)
tf.random.set_seed(42)

In [ ]:
model3 = Sequential()
model3.add(Dense(256,activation='relu',input_dim = X_train_normalized.shape[1]))
model3.add(Dropout(0.2))
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(32,activation='relu'))
model3.add(Dense(9, activation = 'softmax'))

In [ ]:
model3.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)
model3.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

**E. Plot visuals as in Q3.C and share insights about difference observed in both the models.**

In [ ]:
history_3 = model3.fit(X_train_normalized,y_train,batch_size=64,epochs=50,verbose=1,validation_data=(X_test_normalized, y_test))

In [ ]:
plot_tring_accuracy(history_3,'Training Loss','Validation loss','loss')
plot_tring_accuracy(history_3,'Training accuracy','Validation accuracy','accuracy')